### import required module

In [1]:
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
import os
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

### restore model

In [2]:
model_file = 'malel_last-fm_epoch=300'
meta_file_name = 'weights-299.meta'

In [3]:
sess = tf.Session()
saver = tf.train.import_meta_graph('./{}/{}'.format(model_file, meta_file_name))

# モデルの復元
saver.restore(sess,tf.train.latest_checkpoint('./' + model_file + '/'))
# グラフを復元
graph = tf.get_default_graph()

INFO:tensorflow:Restoring parameters from ./malel_last-fm_epoch=300/weights-299


In [13]:
user_embed = graph.get_tensor_by_name("user_embed:0")
entity_embed = graph.get_tensor_by_name("entity_embed:0")
relation_embed = graph.get_tensor_by_name("relation_embed:0")

In [5]:
print(user_embed)

Tensor("user_embed:0", shape=(23566, 64), dtype=float32_ref)


In [6]:
print(entity_embed)

Tensor("entity_embed:0", shape=(106389, 64), dtype=float32_ref)


### Convert tensor to numpy

In [14]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    sess.run(user_embed)
    user_emb_np = user_embed.eval()
    sess.run(entity_embed)
    entity_emb_np = entity_embed.eval()
    sess.run(relation_embed)
    relation_embed_np = relation_embed.eval()

In [15]:
user_num = user_emb_np.shape[0]
print(user_num)

23566


In [16]:
entity_num = entity_emb_np.shape[0]
print(entity_num)

106389


In [17]:
relation_embed_np

array([[-0.01637751, -0.24000359,  0.07463133, ...,  0.03310862,
         0.06729302, -0.21663988],
       [ 0.0717946 ,  0.09735933, -0.02233247, ...,  0.00828215,
         0.09301859, -0.01129991],
       [-0.01506031, -0.21739878,  0.09092385, ..., -0.0920134 ,
         0.05565768, -0.07944979],
       ...,
       [-0.2099327 , -0.18551938,  0.2647716 , ..., -0.0843229 ,
        -0.15165421, -0.00813049],
       [-0.06483795, -0.08999462,  0.21732792, ..., -0.1369419 ,
         0.24128333,  0.091075  ],
       [-0.17682096, -0.0212068 ,  0.12165794, ..., -0.24185516,
         0.20719406, -0.01169294]], dtype=float32)

In [19]:
relation_embed_np.shape[0]

20

###  Dimensional compression

In [ ]:
x = np.concatenate([user_emb_np, entity_emb_np])
print(x.shape)

In [ ]:
X_reduced = TSNE(n_components=2, random_state=0).fit_transform(x)
print(X_reduced.shape)

In [ ]:
x = X_reduced[:, 0].tolist()
y = X_reduced[:, 1].tolist()

user_index = [i for i in  range(user_num)]
entity_index = [i for i in range(entity_num)]

user = ['user']*user_num
entity = ['entity']*entity_num
kind = user + entity

index = user_index + entity_index

dim_2_df = pd.DataFrame(
    data = {
        'id' : index,
        'type' : kind,
        'x' : x,
        'y' : y, 
    },
    columns=['id', 'type', 'x','y']
)

In [ ]:
dim_2_df.head(10)

### save 2dim data

In [ ]:
user_2dim_df = dim_2_df.query('type == "user"')
entity_2dim_df = dim_2_df.query('type == "entity"')

In [ ]:
user_2dim_df.head()

In [ ]:
entity_2dim_df.head()

In [ ]:
save_path = './' + model_file + '/emb_2_dim/'

if not os.path.exists(save_path):
    os.makedirs(save_path)

In [ ]:
user_2dim_df.to_csv(save_path + 'user_emb_2dim.csv')
entity_2dim_df.to_csv(save_path + 'entity_emb_2dim.csv')

### Visualize useing Tensor Board

In [8]:
log_dir = './' + model_file + '/log'

if not os.path.exists(log_dir): 
    os.makedirs(log_dir)

In [9]:
# Embeddings Visualiationにラベルを追加
summary_writer = tf.summary.FileWriter(log_dir)
config = projector.ProjectorConfig()

In [10]:
embedding = config.embeddings.add()

In [11]:
embedding.tensor_name = user_embed.name

In [12]:
projector.visualize_embeddings(summary_writer, config)